In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Semantic search

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("resources/acmecorp-employee-handbook.pdf")

data = loader.load()

print(data)

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-11-20T23:23:16+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-11-20T23:23:16+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resources/acmecorp-employee-handbook.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Employee Handbook\nNon-Disclosure Agreement (NDA) Policy\nEmployees must protect confidential information belonging to the company, its clients, and partners.\nThis includes, but is not limited to, product roadmaps, customer data, internal communications,\nproprietary algorithms, financial information, and unreleased features. Confidential information may not\nbe shared with unauthorized individuals inside or outside the organization. These obligations continue\nafter employment ends.\nWorkplace Conduct Policy\nEmployees must maintain a respectful, professional environment

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

3


Embedding Models: https://docs.langchain.com/oss/python/integrations/text_embedding

In [4]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

embeddings = HuggingFaceEndpointEmbeddings(
    model="Qwen/Qwen3-Embedding-8B",
)

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [6]:
ids = vector_store.add_documents(documents=all_splits)

In [7]:
results = vector_store.similarity_search(
    "How many days of vacation does an employee get in their first year?"
)

print(results[0])

page_content='prohibited. Violations may result in disciplinary action.
Paid Time Off (PTO) Policy
Full■time employees accrue PTO according to the following schedule:  0–1 years of service: 10 days
per year (0.833 days per month)  1–3 years of service: 15 days per year (1.25 days per month)  3+
years of service: 20 days per year (1.67 days per month) PTO may be used for vacation, personal
needs, or illness. Requests should be submitted in advance through the HR system unless related to
an emergency. Employees may carry over up to 5 unused PTO days per calendar year. Extended
absences exceeding 5 consecutive business days require manager approval.
Travel & Expense Policy
Employees may be reimbursed for reasonable and necessary expenses incurred during approved
business travel. This includes transportation, lodging, meals, and incidental expenses within
established limits. Receipts must be submitted within 14 days of travel. First-class travel, personal' metadata={'producer': 'ReportL

## RAG Agent

In [8]:
from langchain.tools import tool

@tool
def search_handbook(query: str) -> str:
    """Search the employee handbook for information"""
    results = vector_store.similarity_search(query)
    return results[0].page_content

In [9]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [10]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[search_handbook],
    system_prompt="You are a helpful agent that can search the employee handbook for information."
    )

In [11]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many days of vacation does an employee get in their first year?")]}
)

In [12]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='How many days of vacation does an employee get in their first year?', additional_kwargs={}, response_metadata={}, id='26f2958b-68df-4cf7-8de1-ec712fa2c1cf'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to answer based on employee handbook. Use search_handbook function with query.', 'tool_calls': [{'id': 'fc_7d4639a0-5dcd-4529-9112-8f6967b3699c', 'function': {'arguments': '{"query":"vacation days first year employee"}', 'name': 'search_handbook'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 152, 'total_tokens': 203, 'completion_time': 0.105519424, 'completion_tokens_details': {'reasoning_tokens': 18}, 'prompt_time': 0.005725508, 'prompt_tokens_details': None, 'queue_time': 0.004555264, 'total_time': 0.111244932}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_178e424650', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs

In [13]:
print(response["messages"][-1].content)

In the first year of employment (0 – 1 year of service), a full‑time employee accrues **10 days of paid time off (PTO) per year** (which works out to about 0.83 day per month). This PTO can be used for vacation, personal needs, or illness.
